In [18]:
# -*- coding: utf-8 -*-

#!wget -O imdb.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vrQ5czMHoO3pEnmofFskymXMkq_u1dPc"
#!unzip imdb.zip
#!pip -q install eli5
#!pip -q install spacy
#!python -m spacy download en

Тут придется авторизировать запрос к гугл драйву, извините.

In [19]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
#!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# downloaded = drive.CreateFile({'id': '1d1-5FwxK53ePwygNWeG7jhsOWZbi5HOv'})
# downloaded.GetContentFile('train_docs.pkl')
#
# downloaded = drive.CreateFile({'id': '1MMOY477t965G0C5DtXeREVp0X85UaNq5'})
# downloaded.GetContentFile('test_docs.pkl')

# Классификация текстов

Начнём с самого простого - анализа тональности текста.
Будем классифицировать отзывы с IMDB на положительные/отрицательные.
Датасет взят с http://ai.stanford.edu/~amaas/data/sentiment/

In [20]:
!head datasets/train_HW6.tsv

"head" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [21]:
import pandas as pd

train_df = pd.read_csv("datasets/train_HW6.tsv", delimiter="\t")
test_df = pd.read_csv("datasets/test_HW6.tsv", delimiter="\t")

print('Train size = {}'.format(len(train_df)))
print('Test size = {}'.format(len(test_df)))

Train size = 25000
Test size = 25000


In [22]:
# from bs4 import BeautifulSoup
# import requests
import json
#
# def getHtmlContent(url, start):
#     out = []
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36',
#     }
#     contents = requests.get(url, headers).text
#     soup = BeautifulSoup(contents, 'lxml')
#
#     for text in soup.find('div', attrs={'class': 'entry-content'}).find_all('p')[start:]:
#         for word in (text).text.lower().split(','):
#             w = word.strip()
#             out.append(w)
#
#     return out
#
#soup_pos = getHtmlContent('https://positivewordsresearch.com/list-of-positive-words/', 3)
#with open('datasets/positive_words.json', 'w', encoding='utf-8') as f:
#    json.dump(soup_pos, f, ensure_ascii=False, indent=4)
#soup_neg = getHtmlContent('https://positivewordsresearch.com/list-of-negative-words/', 1)
#with open('datasets/negative_words.json', 'w', encoding='utf-8') as f:
#    json.dump(soup_neg, f, ensure_ascii=False, indent=4)

#@title Начинаем классифицировать! { vertical-output: true, display-mode: "form" }

def checkAccuracy(  test_df, lastAccur=0,
                    negWord=None,
                    posWord=None,
                    lastPos=None,
                    lastNeg=None):
    if posWord:
        positive_words = (*lastPos, posWord)
    else:
        positive_words = lastPos
    if negWord:
        negative_words = (*lastNeg, negWord)
    else:
        negative_words = lastNeg

    positives_count = test_df.review.apply(lambda text: sum(word in text for word in positive_words))
    negatives_count = test_df.review.apply(lambda text: sum(word in text for word in negative_words))
    is_positive = positives_count > negatives_count
    correct_count = (is_positive == test_df.is_positive).values.sum()

    accuracy = correct_count / len(test_df)
    print('CorrectCount:{} Test accuracy = {:.2%}'.format(correct_count,accuracy))
    if accuracy > lastAccur:# and (accuracy - lastAccur) > 0.005:
        return positive_words, negative_words, accuracy
    else:
        return lastPos, lastNeg, lastAccur

with open('datasets/positive_words.json', 'r') as fp:
    pw = json.load(fp)
with open('datasets/negative_words.json', 'r') as fp:
    nw = json.load(fp)

lastPos = 'love', 'great', 'best', 'wonderful', 'beautiful', 'generous', 'honesty', 'amazed', 'uplift', 'thrive', 'worthiness', 'wholeheartedly', 'to matter', 'anticipation', 'yen', 'contentment', 'exquisite', 'glad', 'grow', 'truth', 'supporting', 'unflappable', 'effectiveness', 'expressing', 'warm', 'authenticity', 'plucky', 'affluence', 'spellbound', 'well-being', 'beauty', 'remarkable', 'freedom', 'progress', 'understanding', 'acclaimed', 'touch', 'forgiveness', 'flexibility', 'equality', 'excellent', 'optimism', 'harmonize', 'piquancy', 'gratitude', 'supercharge', 'sweet', 'dazzled', 'embrace', 'ballistic', 'kindness', 'applaud', 'to be seen', 'awesomeness', 'participation', 'memorable', 'liveliness', 'youthful', 'let go', 'faith', 'courage', 'rapturous', 'pardon', 'flawless', 'nurture', 'duty', 'joy', 'efficacy', 'mastery', 'perseverance', 'admirably', 'amiable', 'sublime', 'loyalty', 'efficient', 'equity', 'acceptance', 'unification'#, "beauty", 'clear' #@param {type:"raw"}
lastNeg = 'worst', 'awful', '1/10', 'crap', 'wful', 'bad', 'distract', 'noxious', 'smells', 'moan', 'stereotypical', 'deceiving', 'incoherent', 'suspicious', 'ghosting', 'bothered', 'sunken', 'slogging', 'spilling', 'grossly', 'encroach', 'cocky', 'screw-up', 'urgent', 'curses', 'rampage', 'crashing', 'fuss', 'crooks', 'tepid', 'bewilderingly', 'lukewarm', 'worse', 'dishonor', 'vomiting', 'unfortunate', 'terrible', 'paucity', 'unsavory', 'hell-bent', 'feign', 'mourn', 'sagging', 'miff', 'bland', 'worn', 'swelled', 'reproach', 'choppy', 'mocked', 'layoff', 'blunders', 'torturous', 'loath', 'disappointingly', 'abnormal', 'abysmally', 'hinder', 'stinks', 'watered-down', 'raving', 'forgetful', 'slug', 'unwatchable', 'decadence', 'butcher', 'f**k' #@param {type:"raw"}
accurNew=0
issetWordPos, issetWordNeg = True, True
import random

def getWordPos(data):
    random.shuffle(data)
    for _ in data:
        yield _

def getWordNeg(data):
    random.shuffle(data)
    for _ in data:
        yield _

gwp = getWordPos(pw)
gwn = getWordNeg(nw)
while issetWordPos and issetWordNeg:
    w = next(gwp)
    if w:
        lastPos, lastNeg, accurNew = checkAccuracy(test_df, accurNew,
                                                  posWord=w,
                                                  lastPos=lastPos,
                                                  lastNeg=lastNeg)
        print(f"{round(accurNew*100,2)}% Positive({w}) {lastPos}")
    else:
        issetWordPos = False
    w = next(gwn)
    if w:
        lastPos, lastNeg, accurNew = checkAccuracy(test_df, accurNew,
                                                  negWord=w,
                                                  lastPos=lastPos,
                                                  lastNeg=lastNeg)
        print(f"{round(accurNew*100, 2)}% Negative({w}) {lastNeg}")
    else:
        issetWordNeg = False

if accurNew > 0.71:
    from IPython.display import Image, display
    display(Image('https://s3.amazonaws.com/achgen360/t/rmmoZsub.png', width=500))

CorrectCount:17296 Test accuracy = 69.18%
69.18% Positive(generous) ('love', 'great', 'best', 'wonderful', 'beautiful', 'generous')
CorrectCount:17309 Test accuracy = 69.24%
69.24% Negative(distract) ('worst', 'awful', '1/10', 'crap', 'wful', 'bad', 'distract')
CorrectCount:17321 Test accuracy = 69.28%
69.28% Positive(honesty) ('love', 'great', 'best', 'wonderful', 'beautiful', 'generous', 'honesty')
CorrectCount:17321 Test accuracy = 69.28%
69.28% Negative(misjudgment) ('worst', 'awful', '1/10', 'crap', 'wful', 'bad', 'distract')
CorrectCount:17309 Test accuracy = 69.24%
69.28% Positive(useful) ('love', 'great', 'best', 'wonderful', 'beautiful', 'generous', 'honesty')
CorrectCount:17326 Test accuracy = 69.30%
69.3% Negative(noxious) ('worst', 'awful', '1/10', 'crap', 'wful', 'bad', 'distract', 'noxious')
CorrectCount:17326 Test accuracy = 69.30%
69.3% Positive(blisscipline) ('love', 'great', 'best', 'wonderful', 'beautiful', 'generous', 'honesty')
CorrectCount:17326 Test accuracy = 69

KeyboardInterrupt: 

**Задание** Придумайте хорошие ключевые слова или фразы и наберите хотя бы 71% точности на тесте (и не забудьте посмотреть на код классификации!)

Ответ: Я построил систему так: Слил с несколькольких страниц через BeautifulSoup положительные и отрицательные слова и фразы, и заредил итерацию для каждого слова из негативных и позитивных, проверяя на всём словаре REVIEW эти слова по одиночке и смотрел на процент, повышает(забрасываю в копилочку) понижает(выбрасываю). Алгоритм можно улучшить(ускорить), просто отбрасывая слово, и не оставляя его в тестируемом массиве.
Вот что получилось: 73.83%

**Задание** Кому-нибудь нравятся эти `<br /><br />`? Лично мне - нет. Напишите регулярку, которая будет их удалять
import re
def change_words_from_dict(text_list, dict_words):
    for regexp in dict_words.keys():
        text_list = re.sub(re.escape(regexp), dict_words[regexp], text_list)